# SQLAlchemy ORM 基础

## Inserting Data 插入数据

要使用 SQLAlchemy 创建一个新记录，我们按照以下步骤操作:

1. Create an object. 创建一个对象
2. Add the object to the session. 将对象添加到会话中
3. Commit the session. 提交会话

在 SQLAlchemy 中，我们使用会话 `session` 与数据库交互。幸运的是，我们不需要手动创建会话，Flask-SQLAlchemy 为我们管理这些。我们以 `db.session` 的形式访问会话对象。它是处理到数据库连接的会话对象。会话对象也是事务的处理程序。默认情况下，`事务隐式启动并保持打开状态`，直到会话被提交或回滚。

~~启动 Python shell 并创建一些模型对象，如下所示:~~
## 补充[不同处]: 我们尝试采用 jupyter notebook 来完成SHELL的功能, 并对多处代码做了合并, 数据库查询直接用mysql客户端的的ASCII 而非图片.

In [1]:
#  注: jupyter notebook 运行这段后会显示每个中间变量的输出, 不然默认只显示最后一个变量的输出或者不显示.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"  

补充[不同处] 创建一个上下文对象,来模拟运行. （不需要，因为是 db = SQLAlchemy(app) 将实例绑定到一个非常特定的 Flask 应用程序）
https://flask-sqlalchemy.palletsprojects.com/en/2.x/api/#configuration


In [2]:
from main2 import app
from flask import url_for

app.config["SQLALCHEMY_ECHO"] = True
app.config['DATABASE_QUERY_TIMEOUT'] = 1
app.config['SQLALCHEMY_RECORD_QUERIES'] = True

# SQLALCHEMY_RECORD_QUERIES
# with app.test_request_context('/api'): # /api is arbitrarily chosen. /API是任意选择的,无关紧要
#     url_for('index')

# app_context = app.app_context()
# app_context.push() 
# 不需要  The difference between the two is that in the first case methods like create_all() and drop_all() will work all the time 
# but in the second case a flask.Flask.app_context() has to exist.



app.config

<Config {'ENV': 'production', 'DEBUG': True, 'TESTING': False, 'PROPAGATE_EXCEPTIONS': None, 'PRESERVE_CONTEXT_ON_EXCEPTION': None, 'SECRET_KEY': 'a really really really really long secret key', 'PERMANENT_SESSION_LIFETIME': datetime.timedelta(days=31), 'USE_X_SENDFILE': False, 'SERVER_NAME': None, 'APPLICATION_ROOT': '/', 'SESSION_COOKIE_NAME': 'session', 'SESSION_COOKIE_DOMAIN': None, 'SESSION_COOKIE_PATH': None, 'SESSION_COOKIE_HTTPONLY': True, 'SESSION_COOKIE_SECURE': False, 'SESSION_COOKIE_SAMESITE': None, 'SESSION_REFRESH_EACH_REQUEST': True, 'MAX_CONTENT_LENGTH': None, 'SEND_FILE_MAX_AGE_DEFAULT': datetime.timedelta(seconds=43200), 'TRAP_BAD_REQUEST_ERRORS': None, 'TRAP_HTTP_EXCEPTIONS': False, 'EXPLAIN_TEMPLATE_LOADING': False, 'PREFERRED_URL_SCHEME': 'http', 'JSON_AS_ASCII': True, 'JSON_SORT_KEYS': True, 'JSONIFY_PRETTYPRINT_REGULAR': False, 'JSONIFY_MIMETYPE': 'application/json', 'TEMPLATES_AUTO_RELOAD': None, 'MAX_COOKIE_SIZE': 4093, 'SQLALCHEMY_DATABASE_URI': 'mysql+pymysql

In [3]:
# 这里我们创建了两个 Category 对象:

from main2 import db, Post, Tag, Category

c1 = Category(name='Python', slug='python')
c2 = Category(name='Java', slug='java')

c1, c2

# 接下来，我们将对象添加到会话中。

db.session.add(c1)
db.session.add(c2)

c1.id, c2.id

(<None:Python>, <None:Java>)

(None, None)

向会话中添加对象实际上并不会将它们写入数据库，它只准备在下一次提交时保存的对象。我们可以通过检查对象的主键来验证这一点。
这两个对象的 id 属性值都是 None。这意味着我们的对象还没有保存在数据库中。

In [4]:
# 我们可以使用 `add_all()` 方法，而不是一次向会话中添加一个对象。`add_all()` 方法接受要添加到会话的对象列表(list)。

db.session.add_all([c1, c1])

# 多次向会话中添加对象不会引发任何错误。在任何时候，您都可以使用 db.session.new 查看会话中的对象。

db.session.new

# 最后，将对象保存到数据库调用 commit ()方法，如下所示:

db.session.commit() 

# 访问 Category 对象的 id 属性现在将返回主键，而不是 None。

c1.id, c2.id

IdentitySet([<None:Python>, <None:Java>])

(1, 2)

此时，mysql 中的 categories 表应该是这样的:
```
mysql> select * from categories;
+----+--------+--------+---------------------+
| id | name   | slug   | created_on          |
+----+--------+--------+---------------------+
|  1 | Python | python | 2021-08-27 07:21:51 |
|  2 | Java   | java   | 2021-08-27 07:21:51 |
+----+--------+--------+---------------------+
2 rows in set (0.00 sec)

```
我们新创建的类别与任何文章都没有关联，因此 c1.posts 和 c2.posts 将返回一个空列表。


In [5]:
c1.posts, c2.posts

([], [])

## 现在让我们创建一些帖子。

In [6]:
p1 = Post(title='Post 1', slug='post-1', content='Post 1', category=c1)
p2 = Post(title='Post 2', slug='post-2', content='Post 2', category=c1)
p3 = Post(title='Post 3', slug='post-3', content='Post 3', category=c2)

# 在创建 Post 对象时，我们不需要传递类别，我们也可以设置如下:
c1,p1
p1.category = c1
p1, p1.category

(<1:Python>, <None:Post 1>)

(<None:Post 1>, <1:Python>)

In [7]:
# 将对象添加到会话并提交。
db.session.add_all([p1,p2,p3])

# db.session.commit  注意必须 带括号
db.session.commit()

```sql
mysql> select * from posts;
+----+--------+--------+---------+---------------------+---------------------+-------------+
| id | title  | slug   | content | created_on          | updated_on          | category_id |
+----+--------+--------+---------+---------------------+---------------------+-------------+
|  1 | Post 1 | post-1 | Post 1  | 2021-08-27 07:35:53 | 2021-08-27 07:35:53 |           1 |
|  2 | Post 2 | post-2 | Post 2  | 2021-08-27 07:35:53 | 2021-08-27 07:35:53 |           1 |
|  3 | Post 3 | post-3 | Post 3  | 2021-08-27 07:35:53 | 2021-08-27 07:35:53 |           2 |
+----+--------+--------+---------+---------------------+---------------------+-------------+
3 rows in set (0.00 sec)

```

In [8]:
# 再次访问 Category 对象的 posts 属性，这次你会得到一个非空的列表，如下所示:
c1.posts , c2.posts

# 从关系的另一边，我们可以使用 Post 对象上的 Category 属性访问文章所属的 Category 对象。
p1.category , p2.category, p3.category

([<1:Post 1>, <2:Post 2>], [<3:Post 3>])

(<1:Python>, <1:Python>, <2:Java>)

请记住，由于 Category 模型中的 `relationship()` 指令，所有这些都成为可能。现在我们的数据库中有三个帖子，但是没有一个与任何标签相关联。

In [9]:
p1.tags, p2.tags, p3.tags

([], [], [])

## 是时候创建一些标签了。在 shell 中创建 Tag 对象如下:

In [18]:
t1 = Tag(name="refactoring", slug="refactoring")
t2 = Tag(name="snippet", slug="snippet")
t3 = Tag(name="analytics", slug="analytics")

db.session.add_all([t1, t2, t3])
db.session.commit()

```sql
mysql> select * from tags;
+----+-------------+-------------+---------------------+
| id | name        | slug        | created_on          |
+----+-------------+-------------+---------------------+
|  1 | refactoring | refactoring | 2021-08-27 07:47:38 |
|  2 | snippet     | snippet     | 2021-08-27 07:47:38 |
|  3 | analytics   | analytics   | 2021-08-27 07:47:38 |
+----+-------------+-------------+---------------------+
3 rows in set (0.00 sec)

```

这段代码创建三个标记对象并将它们提交到数据库。我们的帖子仍然没有连接到任何标签。在这里，我们看看 如何连接一个 `Post` 对象到一个 `Tag` 对象。

In [20]:
app.config["SQLALCHEMY_ECHO"] = True

In [24]:
p1.tags.append(t1) 
p1.tags.extend([t2, t3])
p2.tags.append(t2)
# p3.tags.append([t2, t3])  #'list' object has no attribute '_sa_instance_state'
p3.tags.extend([t2, t3])

db.session.add_all([p1, p2, p3]) 
db.session.new


IdentitySet([])

In [25]:
# 会话本身就像一个集合，所有的项目都可以通过迭代器接口访问:
for obj in db.session:
    print(obj)


<2:Java>
<1:Python>
<3:Post 3>
<1:Post 1>
<2:Post 2>
<2:snippet>
<1:refactoring>
<3:analytics>


In [26]:
# pending objects recently added to the Session
db.session.new

# persistent objects which currently have changes detected
# (this collection is now created on the fly each time the property is called)
db.session.dirty

# persistent objects that have been marked as deleted via session.delete(obj)
db.session.deleted

# dictionary of all persistent objects, keyed on their identity key
db.session.identity_map

IdentitySet([])

IdentitySet([<2:snippet>, <3:Post 3>, <3:analytics>])

IdentitySet([])

In [27]:
p3.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x1f58b0d9a30>,
 'slug': 'post-3',
 'id': 3,
 'updated_on': datetime.datetime(2021, 8, 27, 15, 59, 7),
 'content': 'Post 3',
 'title': 'Post 3',
 'category_id': 2,
 'created_on': datetime.datetime(2021, 8, 27, 15, 59, 7),
 'tags': [<2:snippet>, <3:analytics>, <2:snippet>, <3:analytics>]}

In [34]:
from  flask_sqlalchemy import get_debug_queries

get_debug_queries()

[]

In [35]:
db.session.commit()

In [36]:
get_debug_queries()

[]

这个提交在 `post_tags` 表中添加了以下五条记录。

```sql
mysql> select * from post_tags;
+---------+--------+
| post_id | tag_id |
+---------+--------+
|       1 |      1 |
|       1 |      2 |
|       1 |      3 |
|       2 |      2 |
|       3 |      2 |
|       3 |      3 |
+---------+--------+
6 rows in set (0.00 sec)
```



In [30]:
# 我们的帖子现在与一个或多个标签相关联:

p1.tags, p2.tags, p3.tags
# 可以这样写 
# for p in [p1,p2,p3]: p.tags

([<1:refactoring>, <2:snippet>, <3:analytics>],
 [<2:snippet>],
 [<2:snippet>, <3:analytics>])

[<1:refactoring>, <2:snippet>, <3:analytics>]

[<2:snippet>]

[<2:snippet>, <3:analytics>]

In [22]:
# 反过来，我们可以访问下面这些属于标签的帖子:

t1.posts, t2.posts, t2.posts
#可以这样写 
# for t in [t1,t2,t3]: t.posts

([<1:Post 1>],
 [<1:Post 1>, <2:Post 2>, <3:Post 3>],
 [<1:Post 1>, <2:Post 2>, <3:Post 3>])

In [31]:
# 需要注意的是，与其先提交 Tag 对象，然后再将其与 Post 对象关联，我们可以像下面这样一次完成所有这些操作:

注意，在第11-13行中，我们只是将 `Post` 对象添加到会话中。 `Tag` 和 `Post` 对象通过多对多关系连接。
因此，将 `Post` 对象添加到会话中也会`隐式地`将其关联的 `Tag` 对象添加到会话中。
即使您仍然手动向会话中添加 `Tag` 对象 `db.session.add_all([t1, t2, t3]) `，您也不会得到任何错误。


In [32]:
# 这里不要重复运行, 不然会重复增加 多套tags.
t1 = Tag(name="refactoring", slug="refactoring")
t2 = Tag(name="snippet", slug="snippet")
t3 = Tag(name="analytics", slug="analytics")

p1.tags.append(t1)
p1.tags.extend([t2, t3])
p2.tags.append(t2)
p3.tags.append(t3)

db.session.add(p1)  # 11-13 行  将 `Post` 对象添加到会话中也会`隐式地`将其关联的 `Tag` 对象添加到会话中。
db.session.add(p2)  # 11-13 行
db.session.add(p3)  # 11-13 行

# db.session.add_all([t1, t2, t3])  # 即使您仍然手动向会话中添加 `Tag` 对象 `db.session.add_all([t1, t2, t3]) `，您也不会得到任何错误。

db.session.commit() 

In [22]:
# 如果上面重复运行造成多插入了  可以通过这个删除.
from main2 import post_tags
from sqlalchemy import delete

# 删除post_tags
s = delete(post_tags).where(
    post_tags.c.tag_id > 3
)
db.session.execute(s)
db.session.commit()

# 删除Tag
db.session.query(Tag).filter(
    Tag.id  > 3
).delete(synchronize_session='fetch')

db.session.commit()

# Tag 自增id起始值
sql = 'alter table tags AUTO_INCREMENT=4;'
db.session.execute(sql)
db.session.commit()

t1 = db.session.query(Tag).get(1)
t2 = db.session.query(Tag).get(2)
t3 = db.session.query(Tag).get(3)

t1,t2,t3

0

## 更新数据

要更新对象，只需将其属性设置为新值，将对象添加到会话并提交更改。

In [33]:
p1.content   # initial value

p1.content = "This is content for post 1"   # setting new value
db.session.add(p1)

db.session.commit()

p1.content  # final value

'Post 1'

'This is content for post 1'

## 删除数据

要删除对象，请使用会话对象的 delete ()方法。它接受一个对象，并将其标记为在下次提交时删除。

以下试验 先添加一个SEO的TAG, 再后面再将它删除.

In [56]:
# 创建一个新的临时标签 seo，并将其与后 p1和 p2关联如下:

tmp  = Tag(name='seo', slug='seo')

p1.tags.append(tmp)
p2.tags.append(tmp)

db.session.add_all([p1, p2])
db.session.commit()

这个提交总共添加了3行。一个在 tags 表中，两个在 post_tags 表中。在数据库中，这三行看起来像这样:
```sql
mysql> select * from tags;
+----+-------------+-------------+---------------------+
| id | name        | slug        | created_on          |
+----+-------------+-------------+---------------------+
|  1 | refactoring | refactoring | 2021-08-27 07:47:38 |
|  2 | snippet     | snippet     | 2021-08-27 07:47:38 |
|  3 | analytics   | analytics   | 2021-08-27 07:47:38 |
|  4 | seo         | seo         | 2021-08-27 08:47:51 |  # 这条, 因为前面多加了,又删除
+----+-------------+-------------+---------------------+
4 rows in set (0.00 sec)

mysql> select * from post_tags;
+---------+--------+
| post_id | tag_id |
+---------+--------+
|       1 |      1 |
|       1 |      2 |
|       1 |      3 |
|       2 |      2 |
|       3 |      2 |
|       3 |      3 |
|       1 |      4 | # 这条
|       2 |      4 | # 这条
+---------+--------+
8 rows in set (0.00 sec)
```

让我们现在删除 seo 标签:

In [58]:
tmp

db.session.delete(tmp)
db.session.commit()

<4:seo>

此提交将删除前一步中添加的所有 `三行` (包括tags 和 post_tags)。但是，它不会删除标记所关联的帖子 `post` 。

默认情况下，如果您删除父表中的对象(如 `categories`) ，那么子表中相关对象的外键(如 posts)设置为 `NULL` 。

下面的清单通过创建一个新的 `categories` 对象和一个 post 对象，然后删除该类别对象来演示这种行为:

In [59]:
c4 = Category(name='css', slug='css')

p4 = Post(title='Post 4', slug='post-4', content='Post 4', category=c4)
db.session.add(c4) # 这里只加了C4 但是会附带增加p4
db.session.new  
db.session.commit()

IdentitySet([<None:css>, <None:Post 4>])

这个提交添加了两行，一行在 categories 表中，另一行在 posts 表中。
```sql
mysql> select * from categories;
+----+--------+--------+---------------------+
| id | name   | slug   | created_on          |
+----+--------+--------+---------------------+
|  1 | Python | python | 2021-08-27 07:21:51 |
|  2 | Java   | java   | 2021-08-27 07:21:51 |
|  3 | css    | css    | 2021-08-27 08:58:14 | # 新增加的
+----+--------+--------+---------------------+
3 rows in set (0.00 sec)

mysql> select * from posts;
+----+--------+--------+----------------------------+---------------------+---------------------+-------------+
| id | title  | slug   | content                    | created_on          | updated_on          | category_id |
+----+--------+--------+----------------------------+---------------------+---------------------+-------------+
|  1 | Post 1 | post-1 | This is content for post 1 | 2021-08-27 07:35:53 | 2021-08-27 08:02:25 |           1 |
|  2 | Post 2 | post-2 | Post 2                     | 2021-08-27 07:35:53 | 2021-08-27 07:35:53 |           1 |
|  3 | Post 3 | post-3 | Post 3                     | 2021-08-27 07:35:53 | 2021-08-27 07:35:53 |           2 |
|  4 | Post 4 | post-4 | Post 4                     | 2021-08-27 08:58:14 | 2021-08-27 08:58:14 |           3 | # 新增加的
+----+--------+--------+----------------------------+---------------------+---------------------+-------------+
4 rows in set (0.00 sec)
```

现在让我们看看删除 Category 对象时会发生什么。

In [60]:
db.session.delete(c4)
db.session.commit()

这样提交会从 categories 表中删除 css 类别，并将相关文章的外键(category_id)设置为 NULL。
```sql
mysql> select * from categories;
+----+--------+--------+---------------------+
| id | name   | slug   | created_on          |
+----+--------+--------+---------------------+
|  1 | Python | python | 2021-08-27 07:21:51 |
|  2 | Java   | java   | 2021-08-27 07:21:51 |
+----+--------+--------+---------------------+
3 rows in set (0.00 sec)

mysql> select * from posts;
+----+--------+--------+----------------------------+---------------------+---------------------+-------------+
| id | title  | slug   | content                    | created_on          | updated_on          | category_id |
+----+--------+--------+----------------------------+---------------------+---------------------+-------------+
|  1 | Post 1 | post-1 | This is content for post 1 | 2021-08-27 07:35:53 | 2021-08-27 08:02:25 |           1 |
|  2 | Post 2 | post-2 | Post 2                     | 2021-08-27 07:35:53 | 2021-08-27 07:35:53 |           1 |
|  3 | Post 3 | post-3 | Post 3                     | 2021-08-27 07:35:53 | 2021-08-27 07:35:53 |           2 |
|  4 | Post 4 | post-4 | Post 4                     | 2021-08-27 08:58:14 | 2021-08-27 08:58:14 |        null | # null
+----+--------+--------+----------------------------+---------------------+---------------------+-------------+
4 rows in set (0.00 sec)

在某些情况下，您可能希望在删除父记录后删除所有子记录。我们可以通过向 db.relationship() 指令传递 cascade='all,delete-orphan' 来实现这一点。打开 main2.py 文件并在 Category 模型中修改 db.relationship ()指令，如下所示(突出显示更改) :

flask_app/main2.py
#...
class Category(db.Model):
    #...
    posts = db.relationship('Post', backref='category', cascade='all,delete-orphan')
#...
```

In [ ]:
从现在开始，删除一个类别也会删除与之相关的所有帖子。重新启动 shell 以使更改生效，导入必要的对象，并创建一个新类别和一个帖子，如下所示:

In [61]:
from main2 import db, Post, Tag, Category

c5 = Category(name='css', slug='css')
p5 = Post(title='Post 5', slug='post-5', content='Post 5', category=c5)

db.session.add(c5)
db.session.commit()

下面是数据库应该如何处理这个提交。
```sql
mysql> select * from categories;
+----+--------+--------+---------------------+
| id | name   | slug   | created_on          |
+----+--------+--------+---------------------+
|  1 | Python | python | 2021-08-27 07:21:51 |
|  2 | Java   | java   | 2021-08-27 07:21:51 |
|  4 | css    | css    | 2021-08-27 09:17:54 |
+----+--------+--------+---------------------+
3 rows in set (0.00 sec)

mysql> select * from posts;
+----+--------+--------+----------------------------+---------------------+---------------------+-------------+
| id | title  | slug   | content                    | created_on          | updated_on          | category_id |
+----+--------+--------+----------------------------+---------------------+---------------------+-------------+
|  1 | Post 1 | post-1 | This is content for post 1 | 2021-08-27 07:35:53 | 2021-08-27 08:02:25 |           1 |
|  2 | Post 2 | post-2 | Post 2                     | 2021-08-27 07:35:53 | 2021-08-27 07:35:53 |           1 |
|  3 | Post 3 | post-3 | Post 3                     | 2021-08-27 07:35:53 | 2021-08-27 07:35:53 |           2 |
|  5 | Post 5 | post-5 | Post 5                     | 2021-08-27 09:17:54 | 2021-08-27 09:17:54 |           4 |
+----+--------+--------+----------------------------+---------------------+---------------------+-------------+
4 rows in set (0.00 sec)
```

现在删除类别。

In [63]:
db.session.delete(c5)
db.session.commit()

在这个提交之后，数据库应该是这样的:
```sql
mysql> select * from posts;
+----+--------+--------+----------------------------+---------------------+---------------------+-------------+
| id | title  | slug   | content                    | created_on          | updated_on          | category_id |
+----+--------+--------+----------------------------+---------------------+---------------------+-------------+
|  1 | Post 1 | post-1 | This is content for post 1 | 2021-08-27 07:35:53 | 2021-08-27 08:02:25 |           1 |
|  2 | Post 2 | post-2 | Post 2                     | 2021-08-27 07:35:53 | 2021-08-27 07:35:53 |           1 |
|  3 | Post 3 | post-3 | Post 3                     | 2021-08-27 07:35:53 | 2021-08-27 07:35:53 |           2 |
+----+--------+--------+----------------------------+---------------------+---------------------+-------------+
3 rows in set (0.00 sec)

mysql> select * from categories;
+----+--------+--------+---------------------+
| id | name   | slug   | created_on          |
+----+--------+--------+---------------------+
|  1 | Python | python | 2021-08-27 07:21:51 |
|  2 | Java   | java   | 2021-08-27 07:21:51 |
+----+--------+--------+---------------------+
2 rows in set (0.00 sec)

```


## 查询数据

要查询数据库，我们使用会话对象的 `query()` 方法。`query()`方法返回一个 `flask_sqlalchemy.BaseQuery` 对象，它只是原始 `sqlalchemy.orm.query` 的扩展查询对象。`flask_sqlalchemy.BaseQuery` 对象表示用于查询数据库的 SELECT 语句。下表列出了一些 `flask_sqlalchemy.BaseQuery` 类的常用的方法。

| Method                        | Description                                                  |
| :---------------------------- | :----------------------------------------------------------- |
| `all()`                       | returns the result of the query (represented by `Query`) as a list. 返回查询结果(由 )作为一个列表|
| `count()`                     | returns the total number of records in the query. 返回查询中的记录总数           |
| `first()`                     | returns the first result of the query or `None`, if there are no rows in the result. 返回查询的第一个结果,如果没有发现这样的物体则返回`None` |
| `first_or_404()`              | returns the first result of the query or HTTP 404 Error, if there are no rows in the result. 返回查询的第一个结果,如果没有发现这样的物体则返回 `HTTP 404 Error` |
| `get(pk)`                     | returns an object that matches the given primary key (pk), or `None`, if no such object is found. 返回与给定主键(pk)匹配的对象，如果没有发现这样的物体则返回`None` |
| `get_or_404(pk)`                     | returns an object that matches the given primary key (pk), or `None`, if no such object is found. 返回与给定主键(pk)匹配的对象，如果没有发现这样的物体则返回 `HTTP 404 Error` |
| `filter(*criterion)`          | returns a new `Query` instance after applying the `WHERE` clause to the query. 返回一个新的实例，在应用 `WHERE`子句后|
| `limit(limit)`                | return a new `Query` instance after applying the `LIMIT` clause to the query. 返回一个新的实例，在应用 `LIMIT`(有限数量)子句后|
| `offset(offset)`              | return a new `Query` instance after applying the `OFFSET` clause to the query. 返回一个新的实例，在应用 `OFFSET` (偏移)子句后|
| `order_by(*criterion)`        | return a new `Query` instance after applying `ORDER BY` clause to the query. 返回一个新的实例，在应用 `ORDER BY` 排序子句后|
| `join(*props, **kwargs)`      | return a new `Query` instance after creating SQL INNER JOIN on the query. 返回一个新的实例，在查询上创建 内连接 之后 |
| `outerjoin(*props, **kwargs)` | return a new `Query` instance after creating SQL LEFT OUTER JOIN on the query. 返回一个新的实例，在应用 左外连接 后|
| `group_by(*criterion)`        | return a new `Query` instance after adding `GROUP BY` clause to the query. 返回一个新的实例，在应用 `GROUP BY` (分组统计)子句后|
| `having(criterion)`           | return a new `Query` instance after adding `HAVING` clause to the query. 返回一个新的实例，在应用 `HAVING` (分组筛选) 子句后|

### All ()方法

在其最简单的形式中，query ()方法可以接受一个或多个模型类或列作为参数。

下面的代码返回 posts , categories 和 tags 表中的所有记录。

注: 数据的表现形式是由类的 `__repr__` 函数定义

In [65]:
db.session.query(Post).all()

db.session.query(Category).all()

db.session.query(Tag).all()

[<1:Post 1>, <2:Post 2>, <3:Post 3>]

[<1:Python>, <2:Java>]

[<1:refactoring>, <2:snippet>, <3:analytics>]

要获得用于查询数据库的原始 SQL，只需打印 flask_sqlalchemy.BaseQuery 对象如下:

In [66]:
print(db.session.query(Post))

SELECT posts.id AS posts_id, posts.title AS posts_title, posts.slug AS posts_slug, posts.content AS posts_content, posts.created_on AS posts_created_on, posts.updated_on AS posts_updated_on, posts.category_id AS posts_category_id 
FROM posts


在前面的示例中，数据从表的所有列返回。我们可以通过以下方式将列名显式传递给 query ()方法来防止这种情况:

In [67]:
db.session.query(Post.id, Post.title).all()

[(1, 'Post 1'), (2, 'Post 2'), (3, 'Post 3')]

### 以下做了整合 

In [69]:
# count() 方法返回查询返回的结果数。

db.session.query(Post).count() 
db.session.query(Category).count() 
db.session.query(Tag).count() 

# first() 方法只返回查询的第一个结果，如果查询返回零结果，则返回 None。

db.session.query(Post).first() 
db.session.query(Category).first() 
db.session.query(Tag).first() 

# Get() 方法返回与传递给它的主键相匹配的实例，如果没有找到这样的对象，则返回 None。

db.session.query(Post).get(1) 
db.session.query(Category).get(2) 
db.session.query(Tag).get(3) 

# Get404() 方法返回与传递给它的主键相匹配的实例，如果没有找到这样的对象，则返回 HTTP 404 错误。

db.session.query(Post).get_or_404(1)
db.session.query(Post).get_or_404(100)


3

2

3

<1:Post 1>

<1:Python>

...

<1:Post 1>

<2:Java>

<3:analytics>

<1:Post 1>

NotFound: 404 Not Found: The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.

In [70]:
# Filter() 方法允许我们通过向查询中添加 WHERE 子句来过滤结果。它至少接受一个列、一个运算符和一个值。下面是一个例子:

# 这个查询返回所有标题为“ Post 1”的帖子:
db.session.query(Post).filter(Post.title == 'Post 1').all()
print(db.session.query(Post).filter(Post.title == 'Post 1'))

[<1:Post 1>]

SELECT posts.id AS posts_id, posts.title AS posts_title, posts.slug AS posts_slug, posts.content AS posts_content, posts.created_on AS posts_created_on, posts.updated_on AS posts_updated_on, posts.category_id AS posts_category_id 
FROM posts 
WHERE posts.title = %(title_1)s


In [71]:
# WHERE 子句中的字符串 `%(title_1)s` 是一个占位符，在执行查询时将被实际值替换。
# 我们可以向 filter()方法传递多个过滤器，并且它们将使用 SQL 语言的 `and` 操作符连接在一起。例如:

db.session.query(Post).filter(Post.id >= 1, Post.id <= 2).all()
# 此查询返回主键大于或等于1但小于或等于2的所有职位。它的 SQL 对等语句是:

print(db.session.query(Post).filter(Post.id >= 1, Post.id <= 2))

[<1:Post 1>, <2:Post 2>]

SELECT posts.id AS posts_id, posts.title AS posts_title, posts.slug AS posts_slug, posts.content AS posts_content, posts.created_on AS posts_created_on, posts.updated_on AS posts_updated_on, posts.category_id AS posts_category_id 
FROM posts 
WHERE posts.id >= %(id_1)s AND posts.id <= %(id_2)s


In [72]:
# first_or_404() 与 first ()方法相同，但当查询不返回结果时，它不返回 None，而是返回 HTTP 404 Error。

db.session.query(Post).filter(Post.id > 1).first_or_404()

db.session.query(Post).filter(Post.id > 10).first_or_404().all()

<2:Post 2>

NotFound: 404 Not Found: The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.

In [77]:
# LIMIT ()方法向查询添加 LIMIT 子句。它接受您希望从查询返回的行数。

db.session.query(Post).limit(2).all() # 必须加all(), 不然返回的是一个 `flask_sqlalchemy.BaseQuery` 对象 

db.session.query(Post).limit(2) # 返回的是一个 `flask_sqlalchemy.BaseQuery` 对象 

db.session.query(Post).filter(Post.id >= 2).limit(1).all() 

print(db.session.query(Post).limit(2))

print(db.session.query(Post).filter(Post.id >= 2).limit(1))

# OFFSET ()方法将 OFFSET 子句添加到查询中。它接受偏移量作为参数。它通常与 limit ()子句一起使用。

db.session.query(Post).filter(Post.id > 1).limit(3).offset(1).all()

print(db.session.query(Post).filter(Post.id > 1).limit(3).offset(1))

# ORDER_BY ()方法通过向查询中添加 orderby 子句来对结果进行排序。它接受订单所基于的列名。默认情况下，它按升序排序。

db.session.query(Tag).all()

db.session.query(Tag).order_by(Tag.name).all()

# desc asc 使用 db.desc ()函数按降序排序如下:
db.session.query(Tag).order_by(db.desc(Tag.name)).all()

[<1:Post 1>, <2:Post 2>]

[<2:Post 2>]

SELECT posts.id AS posts_id, posts.title AS posts_title, posts.slug AS posts_slug, posts.content AS posts_content, posts.created_on AS posts_created_on, posts.updated_on AS posts_updated_on, posts.category_id AS posts_category_id 
FROM posts 
 LIMIT %(param_1)s
SELECT posts.id AS posts_id, posts.title AS posts_title, posts.slug AS posts_slug, posts.content AS posts_content, posts.created_on AS posts_created_on, posts.updated_on AS posts_updated_on, posts.category_id AS posts_category_id 
FROM posts 
WHERE posts.id >= %(id_1)s 
 LIMIT %(param_1)s


In [80]:
# join 方法用于创建 sql join。它接受要为其创建 sql join 的表名。

db.session.query(Post).join(Category).all()

print(db.session.query(Post).join(Category))

# Join ()方法通常用于在单个查询中从一个或多个表获取数据。例如:
db.session.query(Post.title, Category.name).join(Category).all()

# 通过链接 JOIN ()方法，我们可以为两个以上的表创建 SQL JOIN:
# db.session.query(Table1).join(Table2).join(Table3).join(Table4).all()



[<1:Post 1>, <2:Post 2>, <3:Post 3>]

SELECT posts.id AS posts_id, posts.title AS posts_title, posts.slug AS posts_slug, posts.content AS posts_content, posts.created_on AS posts_created_on, posts.updated_on AS posts_updated_on, posts.category_id AS posts_category_id 
FROM posts INNER JOIN categories ON categories.id = posts.category_id


[('Post 1', 'Python'), ('Post 2', 'Python'), ('Post 3', 'Java')]

## 让我们通过完成我们的联系表格来结束本课。

回想一下，在 Flask 表单处理课程中，我们创建了一个联系表单来接收用户的反馈。就目前情况而言，`contact()` 视图函数不会将提交的反馈保存到数据库中。它只将反馈打印到控制台。为了将反馈保存到数据库，我们必须首先创建一个新表。打开 main2.py，在 Tag 模型下面添加 Feedback 模型，如下所示:

flask_app/main2.py

```python
#...
class Feedback(db.Model):
    __tablename__ = 'feedbacks'
    id = db.Column(db.Integer(), primary_key=True)
    name = db.Column(db.String(1000), nullable=False)
    email = db.Column(db.String(100), nullable=False)
    message = db.Column(db.Text(), nullable=False)
    created_on = db.Column(db.DateTime(), default=datetime.utcnow)

    def __repr__(self):
        return "<{}:{}>".format(self.id, self.name)
#...
```

重新启动 Python shell 并调用 db 对象的 create_all() 方法来创建反馈表。

In [81]:
from main2 import db

db.create_all()

接下来，修改 contact ()视图函数如下(突出显示更改) :

flask_app/main2.py

```python
#...
@app.route('/contact/', methods=['get', 'post'])
def contact():
    form = ContactForm()
    if form.validate_on_submit():
        name = form.name.data
        email = form.email.data
        message = form.message.data
        print(name)
        print(Post)
        print(email)
        print(message)

        # db logic goes here
        feedback = Feedback(name=name, email=email, message=message)
        db.session.add(feedback)
        db.session.commit()

        print("\nData received. Now redirecting ...")
        flash("Message Received", "success")
        return redirect(url_for('contact'))

    return render_template('contact.html', form=form)
#...
```

启动服务器，访问 http://127.0.0.1:5000/contact/ ，填写表格并提交反馈。

` python main2.py runserver `

在 MYSQL 中提交的反馈应该是这样的:

```sql
mysql> select * from Feedbacks
    -> ;
+----+---------------+----------------+---------+---------------------+
| id | name          | email          | message | created_on          |
+----+---------------+----------------+---------+---------------------+
|  1 | Fluent Python | 4876270@qq.com | success | 2021-08-27 10:02:14 |
+----+---------------+----------------+---------+---------------------+
1 row in set (0.00 sec)
```